<a href="https://colab.research.google.com/github/SanjaPanda/car-plate-detector/blob/main/Model_Training_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import shutil

import os
import sys

import tensorflow as tf

Installing Tensorflow Models.

For more details please follow following link:

https://medium.com/swlh/tensorflow-2-object-detection-api-with-google-colab-b2af171e81cc

https://www.dlology.com/blog/how-to-train-an-object-detection-model-easy-for-free/

In [ ]:
#cd into the TensorFlow directory in your Google Drive
%cd '/content/drive/MyDrive/car-plate-detector/'

try:
  shutil.rmtree('models')
  print('models file deleted')
except:
  pass

#and clone the TensorFlow Model Garden repository
!git clone https://github.com/tensorflow/models.git

/content/drive/MyDrive/car-plate-detector
Cloning into 'models'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 51779 (delta 30), reused 38 (delta 5), pack-reused 51705
Receiving objects: 100% (51779/51779), 568.99 MiB | 19.09 MiB/s, done.
Resolving deltas: 100% (35335/35335), done.
Checking out files: 100% (2250/2250), done.


In [ ]:
#Install some required libraries and tools.

!apt-get install protobuf-compiler python-lxml python-pil
!pip install Cython pandas tf-slim lvis

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-lxml is already the newest version (4.2.1-1ubuntu0.3).
python-pil is already the newest version (5.1.0-1ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 17 not upgraded.


In [ ]:
# Compile the Protobuf libraries.

#cd into 'TensorFlow/models/research'
%cd '/content/drive/MyDrive/car-plate-detector/models/research/'
!protoc object_detection/protos/*.proto --python_out=.
print('Done')

/content/drive/MyDrive/car-plate-detector/models/research
Done


In [ ]:
# Set the environment.

os.environ['PYTHONPATH']+=":/content/drive/MyDrive/car-plate-detector/models/"
sys.path.append("/content/drive/MyDrive/car-plate-detector/models/research")

In [ ]:
# Build and Install setup.py.

%cd '/content/drive/MyDrive/car-plate-detector/models/research/'

!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

/content/drive/MyDrive/car-plate-detector/models/research
Processing /content/drive/MyDrive/car-plate-detector/models/research
     |████████████████████████████████| 9.0MB 12.8MB/s 
     |████████████████████████████████| 1.1MB 56.3MB/s 
     |████████████████████████████████| 2.2MB 23.2MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
     |████████████████████████████████| 61kB 10.0MB/s 
     |████████████████████████████████| 829kB 52.5MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 17.7MB 219kB/s 
     |████████████████████████████████| 153kB 56.3MB/s 
     |████████████████████████████████| 706kB 48.3MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 102kB 14.2MB/s 
     |████████████████████████████████| 1.2MB 45.0MB/s 
     |████████████████████████████████| 645kB 48.6MB/s 
     |████████████████████████████████| 37.6MB 79kB/s 
     |████████████████████████████████| 1

In [ ]:
# Test the installation.

#cd into 'TensorFlow/models/research/object_detection/builders/'
%cd '/content/drive/MyDrive/car-plate-detector/models/research/object_detection/builders/'
!python model_builder_tf2_test.py
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
print('Done')

/content/drive/MyDrive/car-plate-detector/models/research/object_detection/builders
2021-02-13 20:49:29.982560: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
2021-02-13 20:49:33.458995: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-02-13 20:49:33.460169: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-02-13 20:49:33.519984: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-02-13 20:49:33.520620: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 comp

Generate TFrecords that would be used to create model.

In [ ]:
# Generate TFrecords.

#on train data

%cd '/content/drive/MyDrive/car-plate-detector/'

!python3 'scripts/preprocessing/generate_tfrecord.py' \
--csv_input='workspace/training_demo/annotations/carplate_labels_train.csv' \
--output_path='workspace/training_demo/annotations/train.record' \
--image_dir='workspace/training_demo/images/train/'

/content/drive/MyDrive/car-plate-detector
2021-02-13 20:50:05.679105: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Successfully created the TFRecords: /content/drive/MyDrive/car-plate-detector/workspace/training_demo/annotations/train.record


In [ ]:
#on test data

!python3 'scripts/preprocessing/generate_tfrecord.py' \
--csv_input='workspace/training_demo/annotations/carplate_labels_test.csv' \
--output_path='workspace/training_demo/annotations/test.record' \
--image_dir='workspace/training_demo/images/test/'

2021-02-13 20:53:23.206400: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Successfully created the TFRecords: /content/drive/My Drive/car-plate-detector/workspace/training_demo/annotations/test.record



NOTE:

If you haven't downloaded any pre-trained model yet, go back to Step 1 and finish downloading any pre-trained model of your choice.

We are almost ready to start our model training, just a few more steps before we start our model training!

# Step 12- Copying some files

Copy the "model_main_tf2.py" file from "TensorFlow\models\research\object_detection" and paste it into training_demo. We will need this file for training the model.

Copy the "exporter_main_v2.py" file from "TensorFlow\models\research\object_detection" and paste it into training_demo. We will need this file to export the trained model

In [ ]:
%cd '/content/drive/MyDrive/car-plate-detector/'

shutil.copy('models/research/object_detection/model_main_tf2.py', 'workspace/training_demo/', follow_symlinks = True)
shutil.copy('models/research/object_detection/exporter_main_v2.py', 'workspace/training_demo/', follow_symlinks = True)

/content/drive/MyDrive/car-plate-detector


'workspace/training_demo/exporter_main_v2.py'

In [ ]:
%cd '/content/drive/MyDrive/car-plate-detector/'

/content/drive/MyDrive/car-plate-detector


# Start tensorboard

To visualise model training.

In [ ]:
# Start Tensorboard.

#cd into training_demo
%cd '/content/drive/MyDrive/car-plate-detector/workspace/training_demo'

#start the Tensorboard
%load_ext tensorboard
%tensorboard --logdir=models/my_ssd_mobilenet_v2_fpnlite_640x640
# %load_ext tensorboard
# %tensorboard --logdir=models/[name_of_pre-trained-model_you_downloaded]

In [ ]:
# Train the model.

%cd '/content/drive/MyDrive/car-plate-detector/workspace/training_demo/'
#run the cell to start model training 
!python model_main_tf2.py \
--model_dir=models/my_ssd_mobilenet_v2_fpnlite_640x640 \
--pipeline_config_path=models/my_ssd_mobilenet_v2_fpnlite_640x640/pipeline.config \
--alsologtostderr

/content/drive/MyDrive/car-plate-detector/workspace/training_demo
2021-02-13 20:53:53.195014: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-02-13 20:53:55.973456: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-02-13 20:53:55.974349: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-02-13 20:53:56.004604: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-02-13 20:53:56.005196: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-02-13 20:53:

In [ ]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/my_ssd_mobilenet_v2_fpnlite_640x640/pipeline.config --trained_checkpoint_dir ./models/my_ssd_mobilenet_v2_fpnlite_640x640/ --output_directory ./exported-models/my_model_ssd

python3: can't open file 'exporter_main_v2.py': [Errno 2] No such file or directory
